In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
user_df = pd.read_excel('../../data/Visitors Preference Dataset .xlsx')
places_df = pd.read_csv('../../data/Places Dataset Classified Reviews Cleaned Combined .csv')

In [5]:
user_1 = user_df[user_df['User ID'] == 1]
user_1

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell..."


In [6]:
preferred_activities = user_1["Preferred Activities"]
preferred_activities[0]

"['cycling', 'historical monuments', 'village homestays']"

In [7]:
import ast

def convert_string_to_list(input_string):
    return ast.literal_eval(input_string)

preferred_activities_array = convert_string_to_list(preferred_activities[0])
preferred_activities_array

['cycling', 'historical monuments', 'village homestays']

In [8]:
prefered_destinations = user_1["Bucket list destinations Sri Lanka"]
prefered_destinations_array = convert_string_to_list(prefered_destinations[0])
prefered_destinations_array

['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ella', 'Haputale']

In [9]:
for destination in prefered_destinations_array:
    print(destination)

Polonnaruwa
Hatton
Anuradhapura
Ella
Haputale


In [10]:
import pandas as pd

def find_places_based_on_user_preferences(prefered_destinations_array) -> pd.DataFrame:
    pattern = '|'.join(prefered_destinations_array)
    
    filtered_df = places_df[
        places_df['name'].str.contains(pattern, regex=True, na=False) &
        places_df['formatted_address'].str.contains(pattern, regex=True, na=False)
    ]
    
    return filtered_df


In [11]:
input_places = find_places_based_on_user_preferences(prefered_destinations_array)

In [12]:
input_activities = preferred_activities_array
input_activities

['cycling', 'historical monuments', 'village homestays']

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [14]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(places_df['combined_info'])

In [15]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_most_similar_records(input_description, input_activities, tfidf_matrix)-> pd.DataFrame:
    input_activities_str = ' '.join(input_activities)
    input_description = input_description
    
    input_tfidf = tfidf.transform([input_description])
    cosine_similarities = cosine_similarity(input_tfidf, tfidf_matrix).flatten()
    
    related_docs_indices = cosine_similarities.argsort()[:-10:-1]
    return_dict = places_df.iloc[related_docs_indices]
    return_dict['Cosine Similarity'] = cosine_similarities[related_docs_indices]

    return return_dict
    

In [16]:
combined_infos = []

for index, row in input_places.iterrows():
    combined_infos.append(row['combined_info'])
    
combined_infos

['located north central province characterized flat terrain agricultural landscapes former royal capital notable wellpreserved ancient structures relics medieval period home significant buddhist temples stupas showcasing historical religious architecture surrounded lush greenery parakrama samudra reservoir offers scenic beauty biodiversity often features cultural performances reenactments related historical significance variety lodgings available budget midrange options local crafts traditional handicrafts souvenirs available local culinary offerings include traditional sri lankan rice curry dishes',
 'ancient city located north central province characterized flat plains significant water reservoirs one ancient capitals sri lanka known wellpreserved ruins role development buddhism region home numerous stupas temples including significant pilgrimage sites buddhists surrounded green landscapes including sacred bodhi trees network lakes reservoirs cultural festivals local events celebrati

In [17]:
similar_records = pd.DataFrame()

for combined_info in combined_infos:
    new_records = get_most_similar_records(combined_info, input_activities, tfidf_matrix)
    
    if not similar_records.empty:
        new_records = new_records.loc[~new_records.index.isin(similar_records.index)]

    similar_records = pd.concat([similar_records, new_records], ignore_index=False)

# Output the shape of the final DataFrame
print(similar_records.shape)


similar_records.shape


(57, 20)


/var/folders/l0/kmcf3t2n6zbdwxr6qy7v3_980000gn/T/ipykernel_4123/1218007167.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_dict['Cosine Similarity'] = cosine_similarities[related_docs_indices]
/var/folders/l0/kmcf3t2n6zbdwxr6qy7v3_980000gn/T/ipykernel_4123/1218007167.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_dict['Cosine Similarity'] = cosine_similarities[related_docs_indices]
/var/folders/l0/kmcf3t2n6zbdwxr6qy7v3_980000gn/T/ipykernel_4123/1218007167.py:15: SettingWithCopyWarni

(57, 20)

In [18]:
def check_negetive_rate(places_df):
    places = []
    for index, row in places_df.iterrows():
        if row['negetive_rate'] <= 0.2:
            places.append(row)
    return pd.DataFrame(places)

In [19]:
def sort_by_confidence_then_rating(df_places):
    df_places_sorted = df_places.sort_values(by=['Cosine Similarity', 'rating'], ascending=False)
    return df_places_sorted

In [20]:
similar_records = check_negetive_rate(similar_records)

In [21]:
similar_records = sort_by_confidence_then_rating(similar_records)

In [22]:
similar_records.head(5)

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews,activities,Geographical,Historical,Religious,Natural,Entertainment,Accommodation,Shopping,Food,classified_reviews,negetive_rate,combined_info,Cosine Similarity
26,Ella Rock Trailhead,6.857849,81.044023,"Ella, Sri Lanka",4.700000,375.000000,['The Ella Rock Trailhead offers an exhilarati...,"['Hiking', 'Birdwatching', 'Photography']",Situated in the central highlands with steep t...,Rich in colonial history and ancient pathways ...,Close proximity to local temples and shrines r...,"Offers stunning landscapes, diverse flora and ...",Opportunities for eco-tours and guided nature ...,Variety of lodges and guesthouses available ne...,Local markets and souvenir shops in the nearby...,Availability of local cuisine in nearby cafes ...,"[1, 1, 1, 1, 1]",0.0,situated central highlands steep terrain sceni...,1.0
316,Dakkhina Stupa - Anuradhapura,8.341160,80.395259,"Anuradhapura, Sri Lanka",4.600000,183.000000,['The Dakkhina Stupa is a hidden gem in Anurad...,"['Visit the stupa', 'Photography of ancient ar...","Located within a UNESCO World Heritage site, c...","One of the ancient stupas, significant to the ...","A vital pilgrimage site, showcasing Buddhist h...",Surrounded by tranquil landscapes and flora ty...,Educational tours focusing on history and arch...,"Various options available in nearby towns, cat...",Souvenir shops focusing on local crafts and re...,"Local cuisine available in nearby eateries, em...","[1, 1, 1, 1, 1, 1, 1]",0.0,located within unesco world heritage site char...,1.0
278,Dalada Maluva - ( Polonnaruwa),7.947634,81.001233,"Polonnaruwa, Sri Lanka",4.800000,1899.000000,['Dalada Maluva is a mesmerizing experience! T...,"['Cultural Tours', 'Historical Exploration', '...",Located in an ancient city known for its archa...,Served as a significant religious site in the ...,"Houses relics related to the Buddha, attractin...",Surrounded by remnants of ancient architecture...,Engagement in guided tours and educational pro...,Various accommodation options are available ne...,Local handicrafts and souvenirs can be found i...,Traditional food options are available in near...,"[1, 1, 1, 1, 1]",0.0,located ancient city known archaeological sign...,1.0
368,Polonnaruwa Sivan Kovil,7.946288,81.001328,"Polonnaruwa, Sri Lanka",4.800000,146.000000,['Visiting Polonnaruwa Sivan Kovil was a fasci...,"['Temple Visits', 'Cultural Tours', 'Photograp...",Situated in a historically rich region with an...,"Former capital of an ancient kingdom, showcasi...","Hindu temple dedicated to Lord Shiva, featurin...","Surrounded by lush greenery and water bodies, ...",Cultural festivals and religious ceremonies of...,Variety of lodging options available catering ...,Local artisans offer traditional crafts and re...,"Cuisine influenced by local ingredients, featu...","[1, 1, 1, 1, 1]",0.0,situated historically rich region ancient arch...,1.0
78,Haputale,6.765414,80.952565,"Haputale, Sri Lanka",4.459437,638.579937,['Haputale is a beautiful location with stunni...,"['Hiking and trekking', 'Bird watching', 'Tea ...","Located in the central highlands, known for it...",Rich colonial history with remnants of British...,"Home to several Buddhist temples, reflecting t...","Surrounded by lush green hills, diverse flora ...",Offers local cultural performances and festivals.,Variety of lodging options from guesthouses to...,Opportunities to purchase local handicrafts an...,Culinary offerings include local dishes and te...,"[1, 0, 1, 1, 1]",0.2,located central highlands known cool climate s...,1.0


### Inference testing